In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../')

In [6]:
%cd C:\Users\S8kevkla\Downloads\TrainMusic
!git clone https://github.com/abbkevkla/MetalSongGenerator

%cd C:\Users\S8kevkla\Downloads\TrainMusic\MetalSongGenerator\musicautobot-master

C:\Users\S8kevkla\Downloads\TrainMusic
C:\Users\S8kevkla\Downloads\TrainMusic\MetalSongGenerator\musicautobot-master


fatal: destination path 'MetalSongGenerator' already exists and is not an empty directory.


In [3]:
!pip install music21
print("lmao")

lmao


You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
#!pip install pebble
#!pip install jupyter notebook jupyter_contrib_nbextensions
#!pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install fastai

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
from music21 import *
us = environment.UserSettings()
us.create()

UserSettingsException: An environment configuration file already exists; simply set values to modify.

In [ ]:
for key in sorted(us.keys()):
    print(key)

print(us['musescoreDirectPNGPath'])
print(us['musicxmlPath'])

#us['musescoreDirectPNGPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'
us['musicxmlPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'

In [8]:
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *

## MusicTransformer Training

MusicTransformer takes the basic idea of [Language Models](https://en.wikipedia.org/wiki/Language_model) and applies it to Music.  

Given a sequence of notes, predict the next most likely set of notes.

This model is based off of [transformer-XL](https://arxiv.org/abs/1901.02860) and uses fast.ai's [implementation](https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py) of it.

In [5]:
# Location of your midi filesfiles
#midi_path = Path('data/midi/examples')
midi_path = Path('data/midi/Mini_metal')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('data/numpy')
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = 'musicitem_data_save.pkl'

## 1. Gather midi dataset

Make sure all your midi data is in `musicautobot/data/midi` directory

Here's a pretty good dataset with lots of midi data:  
https://www.reddit.com/r/datasets/comments/3akhxy/the_largest_midi_collection_on_the_internet/

Download the folder and unzip it to `data/midi`

## 2. Create dataset from MIDI files

In [6]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

1

In [7]:
processors = [Midi2ItemProcessor()]
data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, bs=2, bptt=12)
data.save(data_save_name)

C:\Users\S8kevkla\AppData\Local\Programs\Python\Python37\lib\site-packages\fastai\data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [8]:
# Show Data
data.train_dl.on_epoch_begin()
x, y = data.one_batch();
x, y

(tensor([[[   0,    0],
          [   1,    0],
          [   8,    0],
          [ 153,    0],
          [  43,   16],
          [ 138,   16],
          [  43,   16],
          [ 138,   16],
          [   8,   16],
          [ 138,   16],
          [  43,   17],
          [ 138,   17]],
 
         [[ 139, 1742],
          [  58, 1744],
          [ 141, 1744],
          [  52, 1744],
          [ 138, 1744],
          [  50, 1744],
          [ 141, 1744],
          [  46, 1744],
          [ 141, 1744],
          [  43, 1744],
          [ 141, 1744],
          [  39, 1744]]]), tensor([[[   1,    0],
          [   8,    0],
          [ 153,    0],
          [  43,   16],
          [ 138,   16],
          [  43,   16],
          [ 138,   16],
          [   8,   16],
          [ 138,   16],
          [  43,   17],
          [ 138,   17],
          [  43,   17]],
 
         [[  58, 1744],
          [ 141, 1744],
          [  52, 1744],
          [ 138, 1744],
          [  50, 1744],
        

## 3. Load Model

In [9]:
batch_size = 1
encode_position = True
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, dl_tfms=dl_tfms)

In [10]:
config = default_config()
config['encode_position'] = encode_position
learn = music_model_learner(data, config=config.copy())

## 4. Train

In [11]:
learn.fit_one_cycle(2)

In [12]:
learn.save('metalexample')

WindowsPath('data/numpy/models/metalexample.pth')

## 5. Predict

---
See [Generate.ipynb](Generate.ipynb) to use a pretrained model and generate better predictions

---

In [19]:
midi_file = Path('data/midi/examples/All I Want For Christmas Is You - Mariah Carey - Pre-Chorus-And-Chorus.mid'); midi_file
#midi_file = Path('data/midi/Metal_midi/Pantera-Walk.mid'); midi_file
item = MusicItem.from_file(midi_file, data.vocab);

In [20]:
item.show('musicxml')

Here's what the seed sounds like:

### Start Predictions:

In [29]:
pred, full = learn.predict(item, n_words=400, temperatures=(1.5,0.8), min_bars=24, top_k=30, top_p=0.7)

Prediction

In [30]:
# Prediction
pred.show('musicxml')

Non midi note detected. Only returning valid portion. Index, seed 21 (200, 2)


In [ ]:
pred.play()